In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm

In [2]:
# 1. Setup: Data Preparation for CIFAR-10
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # InceptionV3 requires 299x299 input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:08<00:00, 20931980.94it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
# 2. Load Pretrained InceptionV3 and Freeze All Layers Except Final Layer
model = torchvision.models.inception_v3(pretrained=True)

# Freeze all the layers
for param in model.parameters():
    param.requires_grad = False

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 160MB/s] 


In [5]:
# Modify the final fully connected layer to match the number of CIFAR-10 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # For CIFAR-10, 10 classes

In [6]:
# 3. Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [7]:
print(device)

cuda


In [8]:
# 4. Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)  # Only training final layer

In [9]:
# 5. Training Loop (Freezing All but Final Layer)
for epoch in range(5):  # Train for a few epochs with frozen layers
    model.train()  # Set model to training mode
    running_loss = 0.0
    progress_bar = tqdm(trainloader, desc=f"Epoch {epoch+1}/5", leave=False)

    for inputs, labels in progress_bar:
        # Move data to GPU
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        if isinstance(outputs, tuple):  # Handle InceptionV3's aux outputs
            outputs = outputs[0]

        # Compute loss
        loss = criterion(outputs, labels)
        loss.backward()

        # Optimize
        optimizer.step()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=running_loss/len(trainloader))

    print(f'Epoch {epoch+1} completed, Loss: {running_loss/len(trainloader):.4f}')


Epoch 1 completed, Loss: 1.3734


Epoch 2 completed, Loss: 0.9810


Epoch 3 completed, Loss: 0.9055


Epoch 4 completed, Loss: 0.8732


Epoch 5 completed, Loss: 0.8477


In [10]:
# 8. Evaluate Model on Test Set
model.eval()  # Set model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        if isinstance(outputs, tuple):
            outputs = outputs[0]
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')

Accuracy on test set: 75.14%


In [11]:
# 6. Fine-tuning: Unfreeze Last Few Layers and Fine-tune Entire Model
# Unfreeze the last few layers of the model (you can adjust which layers)
for name, param in model.named_parameters():
    if 'Mixed_7' in name or 'Mixed_6' in name:  # These are the last two blocks in InceptionV3
        param.requires_grad = True

In [12]:
# Update optimizer to include parameters of the unfrozen layers
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001, momentum=0.9)

In [15]:
# 7. Fine-tuning Training Loop
for epoch in range(5):  # Fine-tune for additional epochs
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(trainloader, desc=f"Fine-tune Epoch {epoch+1}/5", leave=False)

    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        if isinstance(outputs, tuple):  # Handle InceptionV3's aux outputs
            outputs = outputs[0]

        # Compute loss
        loss = criterion(outputs, labels)
        loss.backward()

        # Optimize
        optimizer.step()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=running_loss/len(trainloader))

    print(f'Fine-tune Epoch {epoch+1} completed, Loss: {running_loss/len(trainloader):.4f}')


Fine-tune Epoch 1 completed, Loss: 0.1806


Fine-tune Epoch 2 completed, Loss: 0.1568


Fine-tune Epoch 3 completed, Loss: 0.1339


Fine-tune Epoch 4 completed, Loss: 0.1175


Fine-tune Epoch 5 completed, Loss: 0.1042


In [16]:
# 8. Evaluate Model on Test Set
model.eval()  # Set model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        if isinstance(outputs, tuple):
            outputs = outputs[0]
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')

Accuracy on test set: 93.41%


In [17]:
from torchsummary import summary

# Print the model summary (for input shape of (3, 299, 299) for CIFAR images)
summary(model, (3, 299, 299))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 149, 149]             864
       BatchNorm2d-2         [-1, 32, 149, 149]              64
       BasicConv2d-3         [-1, 32, 149, 149]               0
            Conv2d-4         [-1, 32, 147, 147]           9,216
       BatchNorm2d-5         [-1, 32, 147, 147]              64
       BasicConv2d-6         [-1, 32, 147, 147]               0
            Conv2d-7         [-1, 64, 147, 147]          18,432
       BatchNorm2d-8         [-1, 64, 147, 147]             128
       BasicConv2d-9         [-1, 64, 147, 147]               0
        MaxPool2d-10           [-1, 64, 73, 73]               0
           Conv2d-11           [-1, 80, 73, 73]           5,120
      BatchNorm2d-12           [-1, 80, 73, 73]             160
      BasicConv2d-13           [-1, 80, 73, 73]               0
           Conv2d-14          [-1, 192,

In [19]:
# Save only the model's state_dict (recommended)
torch.save(model.state_dict(), "inception_cifar10.pth")

In [20]:
# Save the entire model (including architecture)
torch.save(model, "inception_cifar10_full.pth")

In [21]:
from google.colab import files

# Download the model weights file (state_dict)
files.download("inception_cifar10.pth")

files.download("inception_cifar10_full.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>